In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [2]:
pip install flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.2/314.2 kB 5.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from flaml import AutoML

In [17]:
train_df = pd.read_csv('/kaggle/input/playground-series-s5e4/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e4/test.csv')

In [6]:
train_df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


In [5]:
submission = pd.read_csv('/kaggle/input/playground-series-s5e4/sample_submission.csv')

In [18]:
X_train = train_df.drop('Listening_Time_minutes', axis=1)
y_train = train_df['Listening_Time_minutes']
X_test = test_df.copy()

In [8]:
automl = AutoML()

In [9]:
automl

AutoML(append_log=False, auto_augment=True, custom_hp={},
       cv_score_agg_func=None, early_stop=False, ensemble=False,
       estimator_list='auto', eval_method='auto', fit_kwargs_by_estimator={},
       force_cancel=False, free_mem_ratio=0, hpo_method='auto',
       keep_search_state=False, learner_selector='sample', log_file_name='',
       log_training_metric=False, log_type='better', max_iter=None,
       mem_thres=4294967296, metric='auto', metric_constraints=[],
       min_sample_size=10000, mlflow_exp_name=None, mlflow_logging=True,
       model_history=False, n_concurrent_trials=1, n_jobs=-1, n_splits=5,
       pred_time_limit=inf, preserve_checkpoint=True, ...)

In [10]:
settings = {
    "time_budget": 600,        # ⏱️ Increase time if needed
    "metric": 'rmse',          # ✅ Optimize for RMSE
    "task": 'regression',
    "log_file_name": "flaml_rmse.log",
    "eval_method": "cv",       # Use cross-validation
    "n_splits": 5,             # 5-fold CV
}

In [11]:
model = automl.fit(X_train=X_train, y_train=y_train, **settings)

[flaml.automl.logger: 04-20 08:22:57] {1728} INFO - task = regression
[flaml.automl.logger: 04-20 08:22:57] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 04-20 08:22:57] {1838} INFO - Minimizing error metric: rmse
[flaml.automl.logger: 04-20 08:22:58] {1955} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'catboost']
[flaml.automl.logger: 04-20 08:22:58] {2258} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 04-20 08:23:01] {2393} INFO - Estimated sufficient time budget=33483s. Estimated necessary time budget=290s.
[flaml.automl.logger: 04-20 08:23:01] {2442} INFO -  at 6.1s,	estimator lgbm's best error=20.9701,	best estimator lgbm's best error=20.9701
[flaml.automl.logger: 04-20 08:23:01] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 04-20 08:23:04] {2442} INFO -  at 9.4s,	estimator lgbm's best error=20.9701,	best estimator lgbm's best error=20.9701
[flaml.automl.logger: 04-20

In [12]:
model

In [19]:
predictions = automl.predict(X_test)

In [20]:
predictions

array([53.55887688, 19.17958906, 49.66566441, ...,  5.26157671,
       75.58707695, 58.66427566])

In [21]:
submission['Listening_Time_minutes'] = predictions

In [22]:
submission.to_csv('using-flaml-automl.csv',index = False)